In [1]:
import leaderbot
import leaderbot as lb
import numpy as np
import matplotlib.pyplot as plt
import texplot
import time
import scipy

In [2]:
data = lb.data.load()

agents = np.array(data['models'])
x = np.array(data['X'])
y = np.array(data['Y'])
n_agents = len(agents)

# Clean empty data
y_sum = np.sum(y, axis=1)
ind_valid = y_sum != 0
x = x[ind_valid]
y = y[ind_valid]

In [3]:
# wins, losses, ties = y[:, 0], y[:, 1], y[:, 2]

# n_wins = np.zeros((n_agents,), dtype=int)
# n_losses = np.zeros((n_agents,), dtype=int)
# n_ties = np.zeros((n_agents,), dtype=int)
# n_games = np.zeros((n_agents,), dtype=int)

# for i in range(n_agents):
#     ind_0 = (x[:, 0] == i)
#     ind_1 = (x[:, 1] == i)
#     n_wins[i] = np.sum(wins[ind_0]) + np.sum(losses[ind_1])
#     n_losses[i] = np.sum(losses[ind_0]) + np.sum(wins[ind_1])
#     n_ties[i] = np.sum(ties[ind_0]) + np.sum(ties[ind_1])

# n_matches = n_wins + n_losses + n_ties
# p_wins = n_wins / n_matches
# p_losses = n_losses / n_matches
# p_ties = n_ties / n_matches

In [9]:
#algs = [davidson, davidson_scaled, davidson_scaled_r, davidson_scaled_rij]
model = lb.DavidsonScaled(data)
model.train()

In [10]:
p_pred = model.infer()

# Evaluate All

In [1]:
import leaderbot as lb
import numpy as np

In [2]:
data = lb.data.load()
training_data, test_data = lb.data.split(data, test_ratio=0.1, seed=20)

In [3]:
from copy import deepcopy

training_data_no_tie = deepcopy(training_data)
test_data_no_tie = deepcopy(test_data)

training_data_no_tie['Y'][:, -1] = 0
test_data_no_tie['Y'][:, -1] = 0

In [4]:
n_tie = 0

models = [
    lb.models.BradleyTerry(training_data),
    lb.models.RaoKupper(training_data, n_tie_factors=n_tie),
    lb.models.Davidson(training_data, n_tie_factors=n_tie)
]

In [13]:
n_tie = 1
n_cov = 2

models = [
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=n_cov),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie),
    lb.models.DavidsonFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie)
]

In [66]:
n_tie = 0
n_cov = 0

models = [
    lb.models.BradleyTerry(training_data_no_tie),
    lb.models.BradleyTerryScaled(training_data_no_tie),
    lb.models.BradleyTerryScaledR(training_data_no_tie),
    # lb.models.BradleyTerryScaledRIJ(training_data, n_tie_factors=n_tie),
    lb.models.BradleyTerryFactor(training_data_no_tie, n_cov_factors=n_cov),
    
    lb.models.RaoKupper(training_data, n_tie_factors=n_tie),
    lb.models.RaoKupperScaled(training_data, n_tie_factors=n_tie),
    lb.models.RaoKupperScaledR(training_data, n_tie_factors=n_tie),
    # lb.models.RaoKupperScaledRIJ(dtraining_data, n_tie_factors=n_tie),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie),
    
    lb.models.Davidson(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonScaled(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonScaledR(training_data, n_tie_factors=n_tie),
    # lb.models.DavidsonScaledRIJ(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie)
]

In [5]:
for model in models:

    name = model.__class__.__name__
    if name.endswith('ScaledRIJ'):
        method = 'L-BFGS-B'
    else:
        method = 'BFGS'

    print(f'training {name:<21} ... ', end='')
    model.train()
    print('done.')

training BradleyTerry          ... done.
training RaoKupper             ... done.
training Davidson              ... done.


In [6]:
metrics = lb.evaluate.model_selection(models, train=False, tie=True, report=True)

+-----------------------+---------+--------+--------+---------+---------+
| model                 | # param | NLL    | CEL    | AIC     | BIC     |
+-----------------------+---------+--------+--------+---------+---------+
| BradleyTerry          |     129 | 0.6552 |    inf |  256.69 | 1036.11 |
| RaoKupper             |     130 | 1.0094 | 1.0094 |  257.98 | 1043.45 |
| Davidson              |     130 | 1.0100 | 1.0100 |  257.98 | 1043.45 |
+-----------------------+---------+--------+--------+---------+---------+


In [7]:
metrics = lb.evaluate.goodness_of_fit(models, train=False, tie=True, metric='MAE', 
                                      density=True, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |             MAE            |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.13  10.22  17.91  12.76 |    inf | 7.2576 |
| RaoKupper             |  4.76   4.85   4.67   4.76 | 0.0335 | 0.9255 |
| Davidson              |  4.78   4.89   4.90   4.86 | 0.0343 | 0.9502 |
+-----------------------+----------------------------+--------+--------+


In [8]:
metrics = lb.evaluate.generalization(models, test_data=test_data, train=False, tie=True,
                                     metric='MAE', density=True, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |             MAE            |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          | 10.45   9.74  17.63  12.61 |    inf | 7.0958 |
| RaoKupper             |  4.51   4.77   4.46   4.58 | 0.0316 | 0.8765 |
| Davidson              |  4.56   4.81   4.68   4.68 | 0.0328 | 0.9122 |
+-----------------------+----------------------------+--------+--------+
